<a href="https://colab.research.google.com/github/GiuliaMaragno/Progetto-Sii-ML/blob/main/AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from matplotlib import pyplot as plt
from tensorflow.keras.models import Model

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential


from keras.layers import Input, Dense, Activation,BatchNormalization, Flatten, Conv2D, MaxPooling2D



import keras.backend as K
K.set_image_data_format('channels_last')

from matplotlib.pyplot import imshow

In [4]:
train_path ='/content/drive/MyDrive/Progetto/kvasir-dataset-v2/Train'
valid_path='/content/drive/MyDrive/Progetto/kvasir-dataset-v2/Valid'
test_path='/content/drive/MyDrive/Progetto/kvasir-dataset-v2/Test'

train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.1, horizontal_flip=True)
valid_datagen = ImageDataGenerator(rescale=1./255)

In [19]:
train_set = train_datagen.flow_from_directory(directory=train_path, target_size=(227, 227), class_mode='categorical', batch_size=32)
valid_set = valid_datagen.flow_from_directory(directory=valid_path, target_size=(227, 227), class_mode='categorical', batch_size=32)
test_set = valid_datagen.flow_from_directory(directory=test_path, target_size=(227,227), class_mode='categorical', batch_size=1, shuffle=False)

Found 5646 images belonging to 8 classes.
Found 1600 images belonging to 8 classes.
Found 800 images belonging to 8 classes.


In [ ]:
print("Batch Size for Input Image : ",train[0][0].shape)
print("Batch Size for Output Image : ",train[0][1].shape)
print("Image Size of first image : ",train[0][0][0].shape)
print("Output of first image : ",train[0][1][0].shape)

In [20]:
def AlexNet(input_shape):
    
    X_input = Input(input_shape)
    
    X = Conv2D(96,(11,11),strides = 4,name="conv0")(X_input)
    X = BatchNormalization(axis = 3 , name = "bn0")(X)
    X = Activation('relu')(X)
    
    X = MaxPooling2D((3,3),strides = 2,name = 'max0')(X)
    
    X = Conv2D(256,(5,5),padding = 'same' , name = 'conv1')(X)
    X = BatchNormalization(axis = 3 ,name='bn1')(X)
    X = Activation('relu')(X)
    
    X = MaxPooling2D((3,3),strides = 2,name = 'max1')(X)
    
    X = Conv2D(384, (3,3) , padding = 'same' , name='conv2')(X)
    X = BatchNormalization(axis = 3, name = 'bn2')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(384, (3,3) , padding = 'same' , name='conv3')(X)
    X = BatchNormalization(axis = 3, name = 'bn3')(X)
    X = Activation('relu')(X)
    
    X = Conv2D(256, (3,3) , padding = 'same' , name='conv4')(X)
    X = BatchNormalization(axis = 3, name = 'bn4')(X)
    X = Activation('relu')(X)
    
    X = MaxPooling2D((3,3),strides = 2,name = 'max2')(X)
    
    X = Flatten()(X)
    
    X = Dense(4096, activation = 'relu', name = "fc0")(X)
    
    X = Dense(4096, activation = 'relu', name = 'fc1')(X) 
    
    X = Dense(8,activation='softmax',name = 'fc2')(X)
    model = Model(inputs = X_input, outputs = X, name='AlexNet')
    return model

In [21]:
alex = AlexNet(train_set[0][0].shape[1:])

In [22]:
alex.summary()

Model: "AlexNet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 227, 227, 3)]     0         
                                                                 
 conv0 (Conv2D)              (None, 55, 55, 96)        34944     
                                                                 
 bn0 (BatchNormalization)    (None, 55, 55, 96)        384       
                                                                 
 activation_10 (Activation)  (None, 55, 55, 96)        0         
                                                                 
 max0 (MaxPooling2D)         (None, 27, 27, 96)        0         
                                                                 
 conv1 (Conv2D)              (None, 27, 27, 256)       614656    
                                                                 
 bn1 (BatchNormalization)    (None, 27, 27, 256)       1024

In [23]:
from tensorflow.python.keras import optimizers

sgd = tf.keras.optimizers.SGD(learning_rate=0.01, weight_decay = 1e-6, momentum = 0.9, nesterov = True)
alex.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics = ['accuracy'])



In [24]:
step_size_train=train_set.n//train_set.batch_size
step_size_valid=valid_set.n//valid_set.batch_size

In [ ]:
alex.fit(x=train_set, steps_per_epoch=step_size_train, validation_data= valid_set, validation_steps=step_size_valid, epochs=50)

In [ ]:
pred = alex.predict(test_set, steps = len(test_set), verbose = 2)

In [ ]:
predicted_class = np.argmax(pred, axis = 1)
true_classes = test_set.classes
class_labels= list(test_set.class_indices.keys())

from sklearn.metrics import classification_report
report = classification_report(true_classes, predicted_class, target_names=class_labels)
print(report)